# 1. 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np

# 2. 데이터 불러오기

In [2]:
df = pd.read_csv('../data/reefer_data_08_celsius.csv')

In [3]:
df.head()

,alarm_codes,ambient_temperature,made_cd,operating_mode,operating_mode_str,reefer_id,return_air_temperature,supply_air_temperature,temperature_setpoint,when_created
0,NaN,23.81,DAIKIN,0.0,DEFROST,CTEU7543690,-18.31,-21.18,-21.12,2022-08-01 00:00:01
1,NaN,24.81,DAIKIN,3.0,THERMO OFF,BMOU8702005,-18.00,-19.31,-18.00,2022-08-01 00:00:17
2,NaN,28.27,CARRIER,3.0,COOL,ARAU2100036,15.93,15.38,9.50,2022-08-01 00:00:24
3,NaN,25.71,CARRIER,3.0,COOL,CRXU5259163,-19.57,-19.42,-20.00,2022-08-01 00:00:28
4,NaN,27.18,DAIKIN,0.0,DEFROST,BMOU8702941,8.50,2.68,2.00,2022-08-01 00:00:37


In [4]:
# CARRIER 데이터만 분리
df_carrier = df.loc[df['made_cd'] == 'CARRIER'].reset_index(drop=True)

In [5]:
df_carrier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152726 entries, 0 to 152725
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   alarm_codes             25692 non-null   object 
 1   ambient_temperature     152670 non-null  float64
 2   made_cd                 152726 non-null  object 
 3   operating_mode          149552 non-null  float64
 4   operating_mode_str      149552 non-null  object 
 5   reefer_id               152726 non-null  object 
 6   return_air_temperature  152726 non-null  float64
 7   supply_air_temperature  152671 non-null  float64
 8   temperature_setpoint    152726 non-null  float64
 9   when_created            152726 non-null  object 
dtypes: float64(5), object(5)
memory usage: 11.7+ MB


In [6]:
# 문자열을 datetime으로 변경
import datetime

df_carrier['when_created'] = df_carrier['when_created'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

## 2-1. 결측치가 가장 적은 컨테이너 찾기

- 10분 단위로 데이터가 수집된다고 가정하면 하루에 144개의 데이터가 수집되어야 한다.
- 8월은 31일까지 있기 때문에 최대 데이터는 144*31 = 4464개이다.
- 따라서 CARRIER의 컨테이너 중 4464개와 가장 근접한 개수를 가진 컨테이너를 찾는 것이 좋겠다.

In [7]:
print(f"8월의 CARRIER사 컨테이너 개수: {df_carrier['reefer_id'].nunique()}개")

8월의 CARRIER사 컨테이너 개수: 64개


In [8]:
print("="*62)
print("데이터의 개수가 4000개 이상인 컨테이너의 데이터 정보를 출력합니다.")
print("="*62)
for reefer_id, length in df_carrier.groupby('reefer_id').size().items():
    if length >= 4000:
        print(f"{reefer_id} 컨테이너의 데이터 개수는 {length}개 입니다.")

데이터의 개수가 4000개 이상인 컨테이너의 데이터 정보를 출력합니다.
WGMU3017777 컨테이너의 데이터 개수는 4213개 입니다.


- 8월의 컨테이너 중 4464개와 가장 근접한 데이터는 1개밖에 없다.
- 그리고 설정 온도가 하필 영하 30도다.

In [9]:
df_4213 = df_carrier.loc[df_carrier['reefer_id'] == 'WGMU3017777'].reset_index(drop=True)

In [10]:
df_4213.head()

,alarm_codes,ambient_temperature,made_cd,operating_mode,operating_mode_str,reefer_id,return_air_temperature,supply_air_temperature,temperature_setpoint,when_created
0,NaN,32.52,CARRIER,3.0,COOL,WGMU3017777,-21.58,-24.42,-30.0,2022-08-01 00:07:50
1,NaN,29.97,CARRIER,5.0,DEFROST,WGMU3017777,-19.06,-21.39,-30.0,2022-08-01 00:18:11
2,NaN,28.96,CARRIER,5.0,DEFROST,WGMU3017777,-13.09,-18.52,-30.0,2022-08-01 00:28:26
3,NaN,28.82,CARRIER,5.0,DEFROST,WGMU3017777,4.27,-17.72,-30.0,2022-08-01 00:38:48
4,NaN,32.15,CARRIER,3.0,COOL,WGMU3017777,-12.50,-14.65,-30.0,2022-08-01 00:49:07


In [11]:
all_day_list = np.arange(1, 32)
exist_day_list = df_4213['when_created'].dt.day.unique()

non_day_list = [x for x in all_day_list if x not in exist_day_list]

for day in non_day_list:
    print(f"8월 {day}일의 데이터가 없습니다.")

8월 2일의 데이터가 없습니다.
8월 4일의 데이터가 없습니다.
8월 6일의 데이터가 없습니다.
8월 13일의 데이터가 없습니다.
8월 23일의 데이터가 없습니다.


- 5일치의 데이터가 없으면 720개의 데이터가 없으므로 4464-720 = 3744개가 있어야 하는데 4213개로 469개의 데이터가 더 수집되었다.
- 따라서 일별 데이터의 개수를 파악할 필요가 있다.

In [12]:
# day 컬럼 생성
df_4213['day'] = df_4213['when_created'].dt.day

In [13]:
df_4213.groupby('day').size()

day
1     145
3     165
5     148
7     140
8     138
9     140
10    153
11    139
12    140
14    140
15    141
16    146
17    212
18    212
19    200
20    166
21    179
22    181
24    175
25    164
26    157
27    148
28    168
29    167
30    167
31    182
dtype: int64

- 확인 결과 대부분의 데이터가 144개보다 더 많은 값을 가지고 있다.
- SHUTDOWN이 발생하면 10분 주기가 아닌 임의로 데이터가 더 수집된다.
- 따라서 해당 컨테이너에 발생한 상태가 어떤 것이 있는지 확인해본다.

In [14]:
df_4213['operating_mode_str'].unique()

array(['COOL', 'DEFROST', 'START UP'], dtype=object)

- SHUTDOWN은 없었다. 어떤 경우에 데이터가 더 많이 수집된 것인지 확인해봐야겠다.
- 1일의 데이터를 살펴보자.

In [15]:
df_4213_day_1 = df_4213.loc[df_4213['day'] == 1]

In [16]:
# 시간 차이 계산을 위한 임의의 데이터 추가
df_4213_day_1 = df_4213_day_1.append(df_4213_day_1.iloc[-1], ignore_index=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_20088\4232301709.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_4213_day_1 = df_4213_day_1.append(df_4213_day_1.iloc[-1], ignore_index=True)


In [17]:
def calc_timestamp(now, future):
    timestamp = []
    for i, j in zip(now, future):
        timestamp.append((((j.hour*60*60) + (j.minute * 60) + (j.second)) - ((i.hour*60*60) + (i.minute * 60) + (i.second))) // 60)
    timestamp.append(0)
    return timestamp

df_4213_day_1['timestamp'] = calc_timestamp(df_4213_day_1['when_created'][:-1], df_4213_day_1['when_created'][1:])

In [18]:
df_4213_day_1['timestamp'].unique()

array([10,  2,  0,  1,  3], dtype=int64)

In [19]:
df_4213_day_1.loc[(df_4213_day_1['timestamp'] == 1) | (df_4213_day_1['timestamp'] == 2) | (df_4213_day_1['timestamp'] == 3)].index

Int64Index([88, 90, 94], dtype='int64')

- 88번 인덱스부터 데이터가 불규칙한 간격으로 수집되었다.

In [20]:
df_4213_day_1.iloc[78:100]

,alarm_codes,ambient_temperature,made_cd,operating_mode,operating_mode_str,reefer_id,return_air_temperature,supply_air_temperature,temperature_setpoint,when_created,day,timestamp
78,NaN,36.01,CARRIER,3.0,COOL,WGMU3017777,-21.49,-24.32,-30.0,2022-08-01 13:32:49,1,10
79,NaN,36.11,CARRIER,3.0,COOL,WGMU3017777,-21.60,-24.37,-30.0,2022-08-01 13:43:07,1,10
80,NaN,35.92,CARRIER,3.0,COOL,WGMU3017777,-21.67,-24.45,-30.0,2022-08-01 13:53:24,1,10
81,NaN,36.10,CARRIER,3.0,COOL,WGMU3017777,-21.76,-24.47,-30.0,2022-08-01 14:03:42,1,10
82,NaN,36.26,CARRIER,3.0,COOL,WGMU3017777,-21.80,-24.53,-30.0,2022-08-01 14:14:01,1,10
83,NaN,36.12,CARRIER,3.0,COOL,WGMU3017777,-21.91,-24.60,-30.0,2022-08-01 14:24:23,1,10
84,NaN,36.17,CARRIER,3.0,COOL,WGMU3017777,-21.91,-24.63,-30.0,2022-08-01 14:34:41,1,10
85,NaN,36.23,CARRIER,3.0,COOL,WGMU3017777,-21.96,-24.64,-30.0,2022-08-01 14:45:01,1,10
86,NaN,32.50,CARRIER,5.0,DEFROST,WGMU3017777,-17.39,-19.59,-30.0,2022-08-01 14:55:19,1,10
87,NaN,31.97,CARRIER,5.0,DEFROST,WGMU3017777,-6.35,-18.17,-30.0,2022-08-01 15:05:41,1,10


- 확인결과 88번부터 이상하게 수집되었다.
- 나머지 데이터도 그럴 가능성이 높다.
- 따라서 다른 데이터를 살펴보도록 한다.

In [21]:
print("="*31)
print("결측일이 5일 이내인 컨테이너 목록")
print("="*31)

use_container_list = []

for reefer_id in df_carrier['reefer_id'].unique():
    container = df_carrier.loc[df_carrier['reefer_id'] == reefer_id]
    all_day_list = np.arange(1, 32)
    exist_day_list = container['when_created'].dt.day.unique()

    non_day_list = [x for x in all_day_list if x not in exist_day_list]

    if len(non_day_list) <= 5:
        print(f"{reefer_id} 컨테이너의 결측일 개수는 {len(non_day_list)}개 입니다.")
        use_container_list.append(container)

결측일이 5일 이내인 컨테이너 목록
CRXU5259163 컨테이너의 결측일 개수는 5개 입니다.
IKRU2200681 컨테이너의 결측일 개수는 5개 입니다.
CRXU5259245 컨테이너의 결측일 개수는 5개 입니다.
CTEU5865614 컨테이너의 결측일 개수는 5개 입니다.
MCLU1037691 컨테이너의 결측일 개수는 5개 입니다.
LXLU5842534 컨테이너의 결측일 개수는 5개 입니다.
RDMU5219631 컨테이너의 결측일 개수는 5개 입니다.
RJCU6995820 컨테이너의 결측일 개수는 5개 입니다.
IKRU2300145 컨테이너의 결측일 개수는 5개 입니다.
UKFU6720496 컨테이너의 결측일 개수는 5개 입니다.
IKRU2300211 컨테이너의 결측일 개수는 5개 입니다.
SDCU6045746 컨테이너의 결측일 개수는 5개 입니다.
CTEU6220543 컨테이너의 결측일 개수는 5개 입니다.
IKRU2201498 컨테이너의 결측일 개수는 5개 입니다.
LBHU2805710 컨테이너의 결측일 개수는 5개 입니다.
RJCU6074831 컨테이너의 결측일 개수는 5개 입니다.
CTEU5050036 컨테이너의 결측일 개수는 5개 입니다.
SMCU6902913 컨테이너의 결측일 개수는 5개 입니다.
CTEU5856290 컨테이너의 결측일 개수는 5개 입니다.
BMOU9004380 컨테이너의 결측일 개수는 5개 입니다.
RDMU5550892 컨테이너의 결측일 개수는 5개 입니다.
RDMU5218830 컨테이너의 결측일 개수는 5개 입니다.
UCSU5853762 컨테이너의 결측일 개수는 5개 입니다.
WGMU3017777 컨테이너의 결측일 개수는 5개 입니다.
TRIU8431795 컨테이너의 결측일 개수는 4개 입니다.
TRIU6614101 컨테이너의 결측일 개수는 5개 입니다.
CTEU3170650 컨테이너의 결측일 개수는 5개 입니다.
SMCU6901080 컨테이너의 결측일 개수는 5개 입니다.
CTEU2659944 컨테이너의 결측일 개수는 5개

In [22]:
print(f"결측일이 5일 이내인 컨테이너는 {len(use_container_list)}개 입니다.")

결측일이 5일 이내인 컨테이너는 31개 입니다.


- 냉동 컨테이너 : 22개
- 냉장 컨테이너 : 9개

In [272]:
# 8월달 시계열 생성 (10분단위) - 총 4464개
month_08 = pd.date_range("2022-08-01", "2022-09-01", freq="10T").to_series().rename('when_created')
month_08.reset_index(drop=True, inplace=True)
month_08 = month_08.iloc[:-1]

# 시계열을 통일한 데이터프레임을 담을 리스트 생성
container_list1 = []

# 컨테이너별로 작업 수행
for i, container in enumerate(use_container_list):
    container = container.set_index('when_created', drop=True)

    # 개별 컨테이너의 시간 단위를 10분 간격으로 조정
    numeric = container.resample('10T').mean().reset_index()
    object = container.resample('10T').first().reset_index()

    # 생성해둔 8월 시계열과 개별 컨테이너 데이터프레임을 병합함
    # 이렇게 하지 않으면 8월20일에 데이터 수집이 종료된 컨테이너는 8월 21일 ~ 8월 31일까지의 데이터가 생성되지 않음
    # 즉, 데이터의 개수가 맞지 않게 됨
    result_numeric = pd.merge(month_08, numeric, on='when_created', how='outer')
    result_object = pd.merge(month_08, object, on='when_created', how='outer')

    # resample 결과 발생하는 결측치를 채움
    # resample 시, 시간밀림 현상 때문에 결측치가 2~3시간 간격으로 결측치가 발생함
    result_numeric = result_numeric.interpolate(method='pad', limit=1)
    result_object = result_object.interpolate(method='pad', limit=1)

    # 동시간대의 값은 평균으로 환산한 데이터는 문자형 변수들이 모두 제거됐으므로 column을 추가해준다.
    result_numeric['alarm_codes'] = result_object['alarm_codes']
    result_numeric['made_cd'] = result_object['made_cd']
    result_numeric['operating_mode_str'] = result_object['operating_mode_str']
    result_numeric['reefer_id'] = result_object['reefer_id']

    # 컨테이너별 데이터를 csv로 변환
    #result.to_csv(f'./data/month08_per_container/container_{i}.csv', index=False)
    container_list1.append(result_numeric)

In [273]:
for reefer in container_list1:
    

,when_created,ambient_temperature,operating_mode,return_air_temperature,supply_air_temperature,temperature_setpoint,alarm_codes,made_cd,operating_mode_str,reefer_id
0,2022-08-01 00:00:00,25.71,3.0,-19.57,-19.42,-20.0,NaN,CARRIER,COOL,CRXU5259163
1,2022-08-01 00:10:00,26.04,3.0,-19.71,-21.75,-20.0,NaN,CARRIER,COOL,CRXU5259163
2,2022-08-01 00:20:00,25.25,6.0,-20.46,-23.10,-20.0,NaN,CARRIER,IDLE,CRXU5259163
3,2022-08-01 00:30:00,25.09,5.0,-18.53,-17.92,-20.0,NaN,CARRIER,DEFROST,CRXU5259163
4,2022-08-01 00:40:00,24.71,5.0,-4.97,-16.30,-20.0,NaN,CARRIER,DEFROST,CRXU5259163
...,...,...,...,...,...,...,...,...,...,...
4459,2022-08-31 23:10:00,19.89,6.0,-20.63,-22.91,-20.0,NaN,CARRIER,IDLE,CRXU5259163
4460,2022-08-31 23:20:00,19.37,6.0,-20.34,-21.17,-20.0,NaN,CARRIER,IDLE,CRXU5259163
4461,2022-08-31 23:30:00,20.17,6.0,-19.96,-19.98,-20.0,NaN,CARRIER,IDLE,CRXU5259163
4462,2022-08-31 23:40:00,19.94,3.0,-19.71,-20.67,-20.0,NaN,CARRIER,COOL,CRXU5259163
